# Q5: MLP with L2 Regularization

In [ ]:
import random
import warnings

warnings.filterwarnings("ignore")
random.seed(1234)

In [ ]:
import csv

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
from models.mlp_model_reg_v3 import MLPReg
from sklearn.metrics import accuracy_score
from training_testing.reg_testing_v3 import plot_decision_boundary
from training_testing.reg_training_v3 import perform_hyperparameter_search, train
from utilities import load_data

## Loading the dataset

In [ ]:
X_train, y_train = load_data("data/center_surround_train.csv")
X_valid, y_valid = load_data("data/center_surround_valid.csv")
X_test, y_test = load_data("data/center_surround_test.csv")

y_train = y_train.reshape(-1, 1)
y_valid = y_valid.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).float()
X_valid_tensor = torch.from_numpy(X_valid).float()
y_valid_tensor = torch.from_numpy(y_valid).float()
X_test_tensor = torch.from_numpy(X_test).float()
y_test_tensor = torch.from_numpy(y_test).float()

In [ ]:
csv_filename = "results/center_surround/reg_l2_hyperparameter_results.csv"
dataset = "center_surround"

## Searching for Hyperparameters

In [ ]:
with open(csv_filename, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(
        [
            "Hidden Layers 1",
            "Hidden Layers 2",
            "Batch Size",
            "Weight Decay",
            "Learning Rate",
            "Epochs",
            "Last Train Loss",
            "Last Validation Loss",
            "Last Train Accuracy",
            "Last Validation Accuracy",
        ]
    )

In [ ]:
hidden_layer_sizes = [5, 10, 15]
batch_sizes = [16, 32]
learning_rates = [0.1, 0.13]
epoch_values = [1500, 2000]
weight_decay_rates = [1e-5, 1e-6]

In [ ]:
perform_hyperparameter_search(
    hidden_layer_sizes,
    batch_sizes,
    learning_rates,
    epoch_values,
    weight_decay_rates,
    X_train_tensor,
    y_train_tensor,
    X_valid_tensor,
    y_valid_tensor,
    csv_filename,
    dataset,
)

## Results from Grid Search

In [ ]:
center_surround_results_df = pd.read_csv(
    f"results/{dataset}/reg_l2_hyperparameter_results.csv"
)

In [ ]:
center_surround_results_df.sort_values(by="Last Validation Loss", ascending=True).head(
    10
)

In [ ]:
lr = 0.1
batch_size = 16
dropout = 0.1
hidden_size1 = 15
hidden_size2 = 15
epochs = 2000
weight_decay = 0.000001

In [ ]:
model = MLPReg(
    input_size=X_train_tensor.shape[1],
    hidden_size1=hidden_size1,
    hidden_size2=hidden_size2,
    output_size=1,
)

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [ ]:
_, _, _, _ = train(
    model,
    criterion,
    optimizer,
    X_train_tensor,
    y_train_tensor,
    X_valid_tensor,
    y_valid_tensor,
    epochs,
    batch_size,
)

In [ ]:
test_pred = model.predict(X_test_tensor)
test_accuracy = accuracy_score(y_test_tensor.squeeze(), test_pred)

print(f"Test accuracy: {test_accuracy}")

## Plotting Decision Boundary

In [ ]:
plot_decision_boundary(model, X_test, y_test, test_pred, dataset)